In [41]:
import pandas as pd

pd.read_csv("evaluation-100.csv").sort_values("factual", ascending=False)

,model,factual,non_factual,unknown,failed,ents_per_sum,sum_with_extrinsic,rouge1,rouge2,rougeL
0,fbs_oracle,0.79,0.21,0.00,10,2.933333,0.46,0.463781,0.238510,0.389488
1,fbs_classifier,0.69,0.31,0.00,13,2.873563,0.41,0.455708,0.234311,0.381817
3,corrector,0.68,0.32,0.00,0,3.070000,0.61,0.461488,0.229356,0.389177
2,baseline,0.66,0.34,0.00,0,3.090000,0.61,0.461872,0.229120,0.389453
4,filtered,0.62,0.37,0.01,0,2.810000,0.58,0.445525,0.207173,0.373429


In [24]:
import pandas as pd

df = pd.read_json("evaluation-100-summaries.json")
df.head()

,fbs_oracle_summary,fbs_oracle_is_non_factual,fbs_oracle_is_factual,fbs_oracle_has_unknown,fbs_oracle_has_failed,fbs_oracle_n_extrinsic,fbs_classifier_summary,fbs_classifier_is_non_factual,fbs_classifier_is_factual,fbs_classifier_has_unknown,...,corrector_is_factual,corrector_has_unknown,corrector_has_failed,corrector_n_extrinsic,filtered_summary,filtered_is_non_factual,filtered_is_factual,filtered_has_unknown,filtered_has_failed,filtered_n_extrinsic
13810760,Former Ospreys scrum-half Mike Phillips has be...,False,True,False,False,1,Former Ospreys scrum-half Mike Phillips has be...,False,True,False,...,True,False,False,1,Wales scrum-half Mike Phillips has been suspen...,False,True,False,False,2
19626825,A public meeting has been held in Flintshire t...,False,True,False,False,0,Hundreds of people have attended a public meet...,True,False,False,...,False,False,False,1,A public meeting has been held to discuss plan...,False,True,False,False,0
21152677,The police ombudsman's office is to resume inv...,False,True,False,False,0,The police ombudsman's office is to resume inv...,False,True,False,...,False,False,False,1,The police ombudsman's office is to resume inv...,False,True,False,False,0
21958547,"The UK's Department for Environment, Food and ...",False,True,False,False,1,"The UK's Department for Environment, Food and ...",False,True,False,...,True,False,False,1,Pesticides used to control pests on crops are ...,False,True,False,False,0
22213813,The remote town of Mashkel is one of the most ...,False,True,False,False,1,The remote town of Mashkel is one of the most ...,False,True,False,...,True,False,False,1,Mashkel is one of the most remote towns in Pak...,False,True,False,False,1


In [38]:
# Helper method
def print_summaries(df, models):
    for sum_id in df.index:
        print(f"---XSUM ID {sum_id}---")
        row = df.loc[sum_id]
        for model in models:
            n_ents = row[model + '_n_extrinsic']
            summary = row[model + '_summary']
            print(f"{model} ({n_ents} extrinsic ents): {[summary]}")
        print()

### Summaries corrected by Oracle

In [48]:
df_inspect = df[
    df.fbs_oracle_is_factual
    & df.baseline_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_oracle"
    ]
)

13
---XSUM ID 19626825---
baseline (1 extrinsic ents): ['Hundreds of people have attended a public meeting in Flintshire to discuss plans to close a community hospital.']
fbs_oracle (0 extrinsic ents): ['A public meeting has been held in Flintshire to discuss plans to close a community hospital.']

---XSUM ID 21152677---
baseline (1 extrinsic ents): ['The police ombudsman is to resume investigations into more than 100 historical complaints against the PSNI.']
fbs_oracle (0 extrinsic ents): ["The police ombudsman's office is to resume investigations into more than 150 historical complaints against the PSNI."]

---XSUM ID 30768958---
baseline (1 extrinsic ents): ['A road in Stafford town centre is to be closed as part of a £1m project to replace water pipes.']
fbs_oracle (0 extrinsic ents): ['A road in Stafford town centre is to be closed as part of a £2.5m project to replace water pipes.']

---XSUM ID 33235610---
baseline (3 extrinsic ents): ['(Close): The pound rose against the euro an

### Summaries messed by FBS Classifier

In [49]:
df_inspect = df[
    df.baseline_is_factual
    & df.fbs_classifier_is_non_factual
]

print(df_inspect.shape)
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_classifier"
    ]
)

(2, 30)
---XSUM ID 33750803---
baseline (1 extrinsic ents): ['At least six people have been killed and more than 30 injured in a suicide bombing in northern Turkey, the army says.']
fbs_classifier (0 extrinsic ents): ['<Failed generation: blocked all beams>']

---XSUM ID 35011005---
baseline (1 extrinsic ents): ["Former Wales captain Martyn Williams says Dan Biggar's decision to sign a new contract with Ospreys will benefit the region."]
fbs_classifier (2 extrinsic ents): ["Dan Biggar's decision to sign a new four-year contract with Ospreys will help the region attract new players, according to former Wales captain Marti Williams."]



In [50]:
df_inspect = df[
    df.baseline_is_non_factual
    & df.fbs_classifier_is_factual
]

print(df_inspect.shape)
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_classifier"
    ]
)

(5, 30)
---XSUM ID 21152677---
baseline (1 extrinsic ents): ['The police ombudsman is to resume investigations into more than 100 historical complaints against the PSNI.']
fbs_classifier (0 extrinsic ents): ["The police ombudsman's office is to resume investigations into more than 150 historical complaints against the PSNI."]

---XSUM ID 30768958---
baseline (1 extrinsic ents): ['A road in Stafford town centre is to be closed as part of a £1m project to replace water pipes.']
fbs_classifier (0 extrinsic ents): ['A road in Stafford town centre is to be closed as part of a £2.5m project to replace water pipes.']

---XSUM ID 33235610---
baseline (3 extrinsic ents): ['(Close): The pound rose against the euro and the dollar on Tuesday, after the Bank of England raised its interest rate to 0.5%.']
fbs_classifier (1 extrinsic ents): ["(Close): The pound rose against the euro and the dollar on Tuesday, following Monday's sharp gains."]

---XSUM ID 33262593---
baseline (0 extrinsic ents): ['A W

### Summaries messed up by Filtered

In [42]:
df_inspect = df[
    df.baseline_is_factual
    & df.filtered_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "baseline",
        "filtered"
    ]
)

13
---XSUM ID 27319850---
baseline (0 extrinsic ents): ['New fathers are often diagnosed with post-traumatic stress disorder (PTSD), a condition that can lead to depression and anxiety.']
filtered (3 extrinsic ents): ['The number of women who suffer from post-traumatic stress disorder (PTSD) is estimated to be one in five women in the UK.']

---XSUM ID 33750803---
baseline (1 extrinsic ents): ['At least six people have been killed and more than 30 injured in a suicide bombing in northern Turkey, the army says.']
filtered (2 extrinsic ents): ['A suicide bomber has attacked a police station in northern Turkey, killing at least five people and injuring more than 20, the Turkish army says.']

---XSUM ID 34886413---
baseline (0 extrinsic ents): ["Singer-songwriter Eska has won this year's Mercury Prize for his debut album Architect."]
filtered (3 extrinsic ents): ['Pioneering singer-songwriter Nick Clegg has won the Mercury Prize for his debut album, The Sound of Silence.']

---XSUM ID 3490

### Summaries messed up by Corrector

In [47]:
df_inspect = df[
    df.baseline_is_factual
    & df.corrector_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "baseline",
        "corrector"
    ]
)

1
---XSUM ID 35011005---
baseline (1 extrinsic ents): ["Former Wales captain Martyn Williams says Dan Biggar's decision to sign a new contract with Ospreys will benefit the region."]
corrector (1 extrinsic ents): ["Former Wales captain Scott Williams says Dan Biggar's decision to sign a new four-year contract with Ospreys will help the region."]

